In [15]:
import sqlite3
with sqlite3.connect('jobs.db') as conn:
    c = conn.cursor()
    c.close()

In [1]:

from config import DB_NAME
import sqlite3, os
from scrapers.db import initialise_db
#initialise database if not exists
if not os.path.exists(DB_NAME):
    initialise_db(DB_NAME) 



In [1]:
import requests
response = requests.get("https://www.rea-group.com/careers/jobs/?team[]=Tech_Other&team[]=Analytics_Other")

In [4]:
with open(r"./sample_pages/Culture Amp response text copy.htm", "w", encoding="utf-8") as f:
    f.write(response.text)

In [2]:
from scrapers.culture_amp_scraper import CultureAmpScraper


scraper = CultureAmpScraper("CultureAmp", "https://www.cultureamp.com/company/careers#open-roles")
jobs = scraper.fetch_jobs()

Skipping non-job listing
Skipping non-job listing


In [6]:
new_jobs = scraper.get_new_jobs(jobs, DB_NAME)


In [9]:
save_jobs = scraper.save_jobs(new_jobs, DB_NAME)

In [3]:
with sqlite3.connect('jobs.db') as conn:

    c = conn.cursor()
    c.execute("SELECT title, location, link FROM job where company = ?", (1,))
    result = c.fetchall() #list of tuples


In [4]:
result

[]

In [5]:
# Append job details to jobs list dataframe
import pandas as pd
jobs_df = pd.DataFrame(columns=["Title", "Location", "Link"])
for job in jobs:
    title, location = job.select('p')
    link = job.get('href')
    jobs_df = pd.concat([jobs_df, pd.DataFrame({"Title": [title.text.strip()], "Location": [location.text.strip()], "Link": [link]})], ignore_index=True)

#save into db
scraper.save_jobs(jobs_df, DB_NAME)